In [1]:
from mmcif.io.PdbxReader import PdbxReader
import pandas as pd

In [2]:
def parse_file(filepath):
    containers = []
    with open(filepath, 'r') as ifh:
        PdbxReader(ifh).read(containers)
        return containers[0]

cif_file = "8zzs.cif"
cif = parse_file(cif_file)

In [3]:
def get_restraints(cif):
    restraints = []
    cross_link_data = cif.getObj('ihm_cross_link_restraint')
    
    for i in range(cross_link_data.getRowCount()):
        data = {
                "asym_id_1": cross_link_data.getValue("asym_id_1", i),
                "seq_id_1": cross_link_data.getValue("seq_id_1", i),
                "atom_id_1" : cross_link_data.getValue("atom_id_1", i),
    
                "asym_id_2": cross_link_data.getValue("asym_id_2", i),
                "seq_id_2": cross_link_data.getValue("seq_id_2", i),
                "atom_id_2" : cross_link_data.getValue("atom_id_2", i),
            
                "distance_threshold": cross_link_data.getValue("distance_threshold", i),
                "restraint_type": cross_link_data.getValue("restraint_type", i),
            }
    
        restraints.append(data)
        df = pd.DataFrame(restraints)
        df['atom_id_1'] = df['atom_id_1'].str.replace('.', 'CA', regex=False)
        df['atom_id_2'] = df['atom_id_2'].str.replace('.', 'CA', regex=False)
        
    return df

restraints = get_restraints(cif)
restraints

,asym_id_1,seq_id_1,atom_id_1,asym_id_2,seq_id_2,atom_id_2,distance_threshold,restraint_type
0,K,44,CA,D,78,CA,28.000,upper bound


In [4]:
from restraint_vis import core

In [5]:
import molviewspec as mvs

In [6]:
builder = mvs.create_builder()
structure = builder.download(url=f"https://pdb-ihm.org/cif/{cif_file}").parse(format="mmcif").assembly_structure(assembly_id="1")

In [7]:
core.visualize_macromolecule(structure, cif)

In [8]:
row = restraints.iloc[0]

restraint_info = dict(
    start_asym_id = row["asym_id_1"],
    start_seq_id = row["seq_id_1"],
    start_atom_id = row["atom_id_1"],
    
    end_asym_id = row["asym_id_2"],
    end_seq_id = row["seq_id_2"],
    end_atom_id = row["atom_id_2"],
    
    distance = row["distance_threshold"],
    restraint_type = row["restraint_type"],
)

core.visualize_restraint(structure, **restraint_info)

In [9]:
builder.save_state(destination="8zz1_restraint.mvsj", title="with restraint")